<a href="https://colab.research.google.com/github/AndyDengFKu/DPA2/blob/main/A2_SEResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models.resnet import ResNet18_Weights
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 定义SE模块
class SEModule(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

# 添加SE模块至ResNet块
def add_se_module_to_resblock(block, channels):
    """Add the SE module to a ResNet block."""
    block.se_module = SEModule(channels)
    block.forward = lambda x: block.se_module(block.orig_forward(x))

# 修改ResNet为SE-ResNet
def modify_resnet_to_seresnet(model):
    for layer in [model.layer1, model.layer2, model.layer3, model.layer4]:
        for block in layer:
            block.orig_forward = block.forward
            add_se_module_to_resblock(block, block.conv2.out_channels)
    return model

# Define the model
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model = modify_resnet_to_seresnet(model)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Load the trained model weights
model_path = ('/content/drive/MyDrive/Colab Notebooks/DeepLearning/A2/2st_model.pth')
model.load_state_dict(torch.load(model_path), strict=False)

criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Defining the Optimiser
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Adding a learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Early stopping parameters
patience = 30
stop_count = 0
best_loss = float('inf')

# Training parameters
num_epochs = 300
validate_every = 1

# Lists to store results for plotting later
train_losses = []
val_losses = []


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 270MB/s]
